In [1]:
import sympy as sm

In [2]:
# Helper function to print the iteration progress.
def progress(i):
    switch ={
        1:"Newton-Raphson iterations begun.",
        10: "Completed 10 iterations...",
        20: "Completed 20 iterations...",
        30: "Completed 30 iterations...",
        40: "Completed 40 iterations...",
        50: "Completed 50 iterations"
    }
    if switch.get(i):
        return switch.get(i)

In [3]:
def newtonRaphson(F, X, X0, xtol=0.001, N=50): 
    
    [dim, _] = X.shape

    if not (isinstance(F, sm.Matrix) and isinstance(X, sm.Matrix)):
        print("Inputs are not matrices")
        return
    
    if not F.shape == X.shape and F.shape == X0.shape:
        print("Dimensions of vectors F, X and X0 do not match")
    
    counter = 1
    while N:       
        Fx = (F).jacobian(X)
        
        X0_dict = {x0: X0[0], x1: X0[1], x2: X0[2]}
        Fe= F.subs(X0_dict)
        
        Fxe = Fx.subs(X0_dict)
        
        if Fxe.det() == 0:
            print("Newton-Raphson failed. The Jacobian matrix evaluated at the estimate is not invertible.")
            return
        else:
            X1 = X0 - Fxe.inv()*Fe # uses Gaussian Elimination by default
        
        X1_dict = {x0: X1[0], x1: X1[1], x2: X1[2]}
        Fe_1= F.subs(X1_dict)
        
        if (Fe_1.norm(1) <= xtol):
            print("Success. Number of iterations: " + str(counter))
            return X1
        
        prog = progress(counter)
        if prog:
            print(prog)
        counter += 1
        N -= 1
        X0 = X1
    print("Newton-Raphson failed to converge with tolerance " + str(xtol) + " after " + str(counter - 1) + " iterations.")
    return

#### 1. Test the algorithm on a system of linear equations:

Define the 3-vector of unkowns $x = \big[x_0\ x_1\ x_2\ \big]^T$:

In [4]:
X, x0, x1, x2 = sm.symbols('X x_0 x_1 x_2')
X = sm.Matrix([x0, x1, x2])
X

Matrix([
[x_0],
[x_1],
[x_2]])

Then define the system of 3 linear equations $\textbf{F(x)}$ to be solved for $\textbf{x}$:

In [5]:
F = sm.Matrix([[8*x0],
           [4*x1],
           [x2 - 4]])
F

Matrix([
[  8*x_0],
[  4*x_1],
[x_2 - 4]])

Then define the vector of estimates:

In [6]:
X0 = sm.Matrix([0.1, 0.1, 0.1])
X0

Matrix([
[0.1],
[0.1],
[0.1]])

Then pass $F$, $X$, and $X_0$ to the `newtonRaphson()` function, which returns the solution:

In [7]:
newtonRaphson(F, X, X0)

Success. Number of iterations: 1


Matrix([
[  0],
[  0],
[4.0]])

#### 2. Test the algorithm on a nonlinear function:

In [8]:
F_trig = sm.Matrix([[sm.cos(x0)*sm.sin(x1)],
                    [(sm.cos(x1))**2],
                    [4*x0*sm.cos(x2)]])
F_trig

Matrix([
[sin(x_1)*cos(x_0)],
[      cos(x_1)**2],
[   4*x_0*cos(x_2)]])

The 3-vector of unknowns $\textbf{x}$ and the solution estimate $\textbf{x}_0$ are already defined above, and can be immediately passed into the algorithm here:

In [9]:
newtonRaphson(F_trig, X, X0)

Newton-Raphson iterations begun.
Success. Number of iterations: 5


Matrix([
[504.225621000936],
[4.73414581917817],
[50339.3098847961]])

#### 3. Test the algorithm on a system of 3rd-order polynomials:

In [10]:
F_poly = sm.Matrix([[2*x0**2 + x1 - x2],
                    [x2**3 - 4],
                    [x1**2 + 4*x1 - 1]])
F_poly

Matrix([
[2*x_0**2 + x_1 - x_2],
[          x_2**3 - 4],
[  x_1**2 + 4*x_1 - 1]])

In [11]:
X0 = sm.Matrix([0.1, 0.1, 0.1])
newtonRaphson(F_poly, X, X0)

Newton-Raphson iterations begun.
Completed 10 iterations...
Success. Number of iterations: 15


Matrix([
[0.821989648050178],
[ 0.23606797749979],
[ 1.58740158419554]])

#### 4a. Test the algorithm with a trig function that doesn't converge after 1000 iterations using $x_0 = [0.1\ 0.1\ 0.1]^T$ as the starting estimates:

In [12]:
F_trig_bad = sm.Matrix([[sm.cos(x0)*sm.tan(x1)],
                    [(sm.cos(x1))**2],
                    [4*x0*sm.cos(x2)]])
X0 = sm.Matrix([0.1, 0.1, 0.1])

In [13]:
newtonRaphson(F_trig_bad, X, X0)

Newton-Raphson iterations begun.
Completed 10 iterations...
Completed 20 iterations...
Completed 30 iterations...
Completed 40 iterations...
Completed 50 iterations
Newton-Raphson failed to converge with tolerance 0.001 after 50 iterations.


#### 4b. Test the algorithm with a trig function where the determinant of the Jacobian $|\textbf{F}_x\big(\textbf{x}^j\big)|=0$ and therefore equation (2.2.60) cannot be evaluated:

In [14]:
F_trig_bad = sm.Matrix([[sm.cos(x0)*sm.tan(x1)],
                    [(sm.cos(x1))**2],
                    [4*x0*sm.cos(x2)]])
X0_bad = sm.Matrix([0.1, 0, 0.1])

To show that the determinant of the Jacobian is zero, part of the Newton-Raphson algorithm is computed below before calling the algorithm.

In [15]:
Fx = (F_trig_bad).jacobian(X)
        
X0_dict = {x0: X0_bad[0], x1: X0_bad[1], x2: X0_bad[2]}
Fe= F.subs(X0_dict)
        
Fxe = Fx.subs(X0_dict)
Fxe

Matrix([
[              0, 0.995004165278026,                   0],
[              0,                 0,                   0],
[3.9800166611121,                 0, -0.0399333666587313]])

In [16]:
Fxe.det()    # determinant = 0

0

In [17]:
newtonRaphson(F_trig_bad, X, X0_bad)

Newton-Raphson failed. The Jacobian matrix evaluated at the estimate is not invertible.


#### 4c. Test the algorithm with a trig function where the estimate leads to a solution when the error tolerance is increased to 0.5, but does not converge when the error tolerance is set to 0.1:

In [18]:
X0_good = sm.Matrix([0.5, 0.5, 0.5])
xtol_good = 0.5
newtonRaphson(F_trig_bad, X, X0_good, xtol_good)

Newton-Raphson iterations begun.
Completed 10 iterations...
Completed 20 iterations...
Completed 30 iterations...
Completed 40 iterations...
Success. Number of iterations: 46


Matrix([
[58.1194640914112],
[1.57079632679489],
[23.5619449019235]])

In [19]:
X0_good = sm.Matrix([0.5, 0.5, 0.5])
xtol_bad = 0.1
newtonRaphson(F_trig_bad, X, X0_good, xtol_bad)

Newton-Raphson iterations begun.
Completed 10 iterations...
Completed 20 iterations...
Completed 30 iterations...
Completed 40 iterations...
Completed 50 iterations
Newton-Raphson failed to converge with tolerance 0.1 after 50 iterations.
